[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# Introducción a *Dask*.

## Contexto: Del análisis local al escalado distribuido

En los capítulos anteriores aprendiste sobre **Polars** y **PyArrow**, que te ofrecen capacidades poderosas para analizar datos en una máquina. Sin embargo, cuando tus datos crecen más allá de lo que pueda caber en la memoria de una computadora, necesitas una estrategia diferente.

### ¿Polars o Dask?

| Aspecto | **Polars** | **Dask** |
|---------|-----------|----------|
| **Tamaño de datos** | Hasta ~100 GB | >100 GB (terabytes) |
| **Localidad de ejecución** | Una máquina | Múltiples máquinas (clúster) |
| **Velocidad (single-machine)** | ⚡ 3-10x más rápido que Pandas | 🚀 Distribuida, puede ser más lenta por overhead |
| **Facilidad de uso** | 😊 API simple y moderna | 😐 Más complejo, requiere cluster |
| **Casos de uso** | ML local, analítica exploratoria | Big Data, ETL distribuido, producción |
| **Integración distribuida** | Reciente, experimental | Madura, bien establecida |

**Regla de oro:**
- **Polars** cuando tus datos caben en memoria (y quieres máxima velocidad)
- **Dask** cuando tus datos no caben en una máquina

---


Las bibliotecas de *Scipy* tienen limitaciones en cuanto a su capacidad de escalar de forma horizontal y aún cuando son capaces de realizar *multithreading* para procesamiento en paralelo, están restringidas a la cantidad de recursos disponibles de la máquina de las que son ejecutadas.

[*Dask*](https://dask.org/) es una biblioteca general para cómputo paralelo que permite escalar sus operaciones por medio de clústers (grupos de equipos de cómputo que trabajan de forma coordinada).

*Dask* consta de:

* Un calendarizador de tareas dinámico (*dynamic task scheduler*).
* Una colección de bibliotecas optimizadas para *Big Data*, con interfaces que extienden a*Numpy* y *Pandas*.

https://docs.dask.org/en/stable/

https://tutorial.dask.org/

In [ ]:
pip install dask

## Principales paquetes de *Dask*.

<img src="img/arquitectura_dask.png" width=75%>

### Paquetes de colecciones de datos de *Dask*.

* ```dask.array```, el cual contiene una biblioteca para manejo de arreglos similar a la de *Numpy*. Por convención, este módulo se importa como ```da```. La documentación de este paquete puede consultarse en:
 * https://docs.dask.org/en/stable/array.html
* ```dask.dataframe```, el cual contiene una biblioteca para manejo de *dataframes* similar a la de *Pandas*. Por convención, este módulo se importa como ```dd```. La documentación de este paquete puede consultarse en:
 * https://docs.dask.org/en/stable/dataframe.html
* ```dask.bags```, el cual contiene una biblioteca para manejo de *bags*, las cuales son estructuras de datos que pueden contener datos semi-estructurados y estructurados. Por convención este módulo se importa como ```db```. La documentación de este paquete puede consultarse en:
https://docs.dask.org/en/stable/bag.html

### Evaluación perezosa (*lazy*) con el método ```compute()```.

In [ ]:
import dask.dataframe as dd

In [ ]:
df = dd.read_csv('data/data_covid.csv')

In [ ]:
df

In [ ]:
df.compute()

In [ ]:
type(df["Nacional"])

In [ ]:
df["Nacional"].compute()

In [ ]:
df.loc[df["Nacional"] > 50000].loc[:, ['index', 'Nacional']]

In [ ]:
df.loc[df["Nacional"] > 50000].loc[:, ['index', 'Nacional']].compute()

### Persistencia en memoria con `persist()`.

Con evaluación perezosa, cada llamada a `compute()` **recalcula todo el grafo** desde el origen. Cuando el mismo DataFrame se reutiliza en múltiples operaciones, esto implica releer y reprocesar los datos innecesariamente.

`persist()` ejecuta el grafo **una sola vez** y retiene los resultados en memoria, comportándose como un caché distribuido.

| Operación | Comportamiento |
|-----------|----------------|
| `compute()` | Ejecuta el grafo y devuelve el resultado a Python |
| `persist()` | Ejecuta el grafo y retiene el resultado en los workers |

https://docs.dask.org/en/stable/api.html#dask.dataframe.DataFrame.persist

In [ ]:
import dask.dataframe as dd

df = dd.read_csv('data/data_covid.csv')

# Sin persist: cada operación relee y recalcula desde disco
resultado_a = df[df['Nacional'] > 50000].compute()
resultado_b = df['Nacional'].mean().compute()

# Con persist: los datos se cargan una sola vez en memoria
df_cache = df.persist()

# Las siguientes operaciones son más rápidas — los datos ya están en memoria
resultado_a = df_cache[df_cache['Nacional'] > 50000].compute()
resultado_b = df_cache['Nacional'].mean().compute()

print(f"Media Nacional: {resultado_b:.2f}")
print("Resultados calculados con datos en caché.")

### Bibliotecas de *Dask*.

* ```dask.delayed```. Esta biblioteca permite procesar colecciones basadas en *Python* de forma paralela.
 * https://docs.dask.org/en/stable/delayed.html
* ```dask.futures```. Es una implementación de [```concurrent.futures```](https://docs.python.org/3/library/concurrent.futures.html) de *Python* optimizado para correr en un cluster. La documentación de este paquete puede consultarse en:
 * https://docs.dask.org/en/stable/futures.html

## Despliegue de un cluster con ```Dask.Distributed```.

*Dask* puede ser desplegado en clusters mediante el uso de varios equipos *workers* gestionados por un *scheduler*.


https://distributed.dask.org/en/stable/

<img src="img/dask_cluster.png" width=45%>

In [ ]:
!pip install "bokeh>=2.4.2, <3"
!pip install dask distributed --upgrade

In [ ]:
!dask scheduler

## Integración: Polars → Dask (Escalado desde análisis local)

Un patrón común en análisis de datos es:
1. **Explorar** datos con **Polars** en tu máquina (rápido)
2. **Escalar** a **Dask** cuando necesites procesamiento distribuido

Dask puede leer archivos Parquet generados por Polars de forma eficiente:


In [ ]:
# Ejemplo: Leer Parquet escrito por Polars con Dask
# df_dask = dd.read_parquet('data/mi_archivo.parquet')
# 
# Ventajas:
# - Parquet preserva tipos de datos de Polars
# - Dask puede leer particiones en paralelo
# - Sin conversión intermedia necesaria


## Procesamiento de datos particionados.

El **particionado Hive-style** organiza los archivos Parquet en subdirectorios según el valor de una columna:

```
datos/
  region=norte/
    part.0.parquet
  region=sur/
    part.0.parquet
  region=este/
    part.0.parquet
```

Dask puede leer únicamente las particiones necesarias (*partition pruning*), evitando leer archivos irrelevantes. Esto reduce significativamente el I/O en grandes volúmenes de datos.

https://docs.dask.org/en/stable/dataframe-parquet.html

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Datos de ejemplo con columna de partición
df_ejemplo = pd.DataFrame({
    'region':   ['norte', 'norte', 'sur', 'sur', 'este'],
    'producto': ['A', 'B', 'A', 'C', 'B'],
    'ventas':   [100, 200, 150, 300, 250]
})

ddf = dd.from_pandas(df_ejemplo, npartitions=1)

# Escribir particionado por 'region' (Hive-style)
ddf.to_parquet('data/ventas_particionadas/', partition_on=['region'], overwrite=True)
print("✓ Datos escritos con particionado por región")

In [ ]:
# Leer solo la región 'norte' — Dask no toca los demás archivos (partition pruning)
df_norte = dd.read_parquet(
    'data/ventas_particionadas/',
    filters=[('region', '==', 'norte')]
)

print(f"Particiones leídas: {df_norte.npartitions}")
print(df_norte.compute())

### Caso de uso: De Polars a Dask

```python
# Paso 1: Procesar con Polars (rápido, single-machine)
import polars as pl
df_polars = pl.read_csv('datos_grandes.csv')
df_procesado = df_polars.filter(pl.col('fecha') > '2023-01-01')
df_procesado.write_parquet('datos_procesados.parquet')

# Paso 2: Distribuir con Dask si es necesario
import dask.dataframe as dd
df_dask = dd.read_parquet('datos_procesados.parquet')
resultado = df_dask.groupby('categoria').agg({'valor': 'mean'}).compute()
```

**Ventajas de este enfoque:**
- Polars maneja el preprocesamiento rápido
- Dask escala el procesamiento distribuido
- Parquet es el estándar de facto para datos columnares


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2017-2026.</p>